In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
train=pd.read_csv('/content/drive/MyDrive/Workspace/hasocpaper/trainset.csv')
test=pd.read_csv('/content/drive/MyDrive/Workspace/hasocpaper/testset.csv')

In [ ]:
train.dropna(inplace=True)
test.dropna(inplace=True)

TRAINING AND TESTING

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sn
import pickle

%matplotlib inline

#Import module to split the datasets
from sklearn.model_selection import train_test_split
# Import modules to evaluate the metrics
from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc

In [ ]:
root_folder='/content/drive/MyDrive/Workspace/Alta/'
data_folder_name='dataset'
glove_filename='glove.840B.300d.txt'

# Variable for data directory
DATA_PATH = os.path.abspath(os.path.join(root_folder, data_folder_name))
glove_path = os.path.abspath(os.path.join(DATA_PATH, glove_filename))

# # Both train and test set are in the root data directory
# train_path = DATA_PATH
# test_path = DATA_PATH

# #Relevant columns
# TEXT_COLUMN = 'features'
# TARGET_COLUMN = 'label'

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec

#glove_input_file = glove_filename
word2vec_output_file = glove_filename+'.word2vec'
glove2word2vec('/content/drive/MyDrive/Workspace/Alta/glove.840B.300d.txt', word2vec_output_file)

(2196017, 300)

In [ ]:
from gensim.models import KeyedVectors
# load the Stanford GloVe model
word2vec_output_file = glove_filename+'.word2vec'
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [ ]:
class Word2VecVectorizer:
  def __init__(self, model):
    print("Loading in word vectors...")
    self.word_vectors = model
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    # determine the dimensionality of vectors
    v = self.word_vectors.get_vector('king')
    self.D = v.shape[0]

    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Workspace/hasoc/trainpSet.csv')

In [ ]:
df

,features,task_1,task_2
0,wealth if you made it through this amp were no...,HOF,PRFN
1,Technically that still turning back the clock...,HOF,OFFN
2,VMBJP BJP Bengal India narendramodi JPNadda Am...,NOT,NONE
3,krtoprak yigit Soldier of Japan Who has dick head,HOF,OFFN
4,blueheartedly You be better off asking who DO...,HOF,OFFN
...,...,...,...
3838,BBCNews Let the dog deal with wanker once he ...,HOF,PRFN
3839,India has suffered lot That Chinese bastard s...,HOF,HATE
3840,People didn give seats majority to BJP see Be...,HOF,HATE
3841,KanganaTeam This is such vile xenophobic and ...,HOF,PRFN


In [ ]:
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
df['new_label1']= label_encoder.fit_transform(df['task_1'].astype(str))
# df['new_label2']= label_encoder.fit_transform(df['task_2'].astype(str))

In [ ]:
from sklearn.model_selection import train_test_split

X_train = train.tweet.values
y_1 = train.label.values
# y_2 = df.new_label2.values

# X_train, X_val, y_train, y_val =\
#     train_test_split(X, y, test_size=0.2, random_state=2020)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.svm import SVC,LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier,GradientBoostingClassifier
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
X_train

array(["as a woman you shouldn't complain about cleaning up your house. amp as a man you should always take the trash out...",
       'boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place',
       'dawg you ever fuck a bitch and she start to cry? you be confused as shit',
       ...,
       "people didn't give 300 seats majority to bjp to see bengal burning ... if they can't fix this, they hv no right to continue in office... don't take our votes for granted ...",
       'this is such a vile, xenophobic and uneducated comment... i m struggling to believe someone thinks like this, let alone posted this? daylight islamophobia and it should be stopped. elections',
       'chinese virus wuhan virus is the correct name for the pandemic . shameless'],
      dtype=object)

In [ ]:
for sentence in X_train:
    # print(type(sentence))
    if(type(sentence)!=str):
        print(sentence)
    #   tokens = sentence.split()

In [ ]:
vectorizer = Word2VecVectorizer(model)
# Get the sentence embeddings for the train dataset
Xtrain = vectorizer.fit_transform(X_train)
Ytrain1 = y_1
# Ytrain2=y_2
# Get the sentence embeddings for the test dataset
# Xtest = vectorizer.transform(X_val)
# Ytest = y_val

Loading in word vectors...
Finished loading in word vectors
Numer of samples with no words found: 161 / 76578


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
# # create the model, train it, print scores
clf1 = GaussianNB().fit(Xtrain, Ytrain1)
# clf2 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(Xtrain, Ytrain)
clf3 = LogisticRegression(random_state=0).fit(Xtrain, Ytrain1)
clf4 = MLPClassifier(random_state=1, max_iter=300).fit(Xtrain, Ytrain1)
# clf5 = make_pipeline(StandardScaler(), SVC(gamma='auto')).fit(Xtrain, Ytrain1)
clf6 = VotingClassifier(estimators=[ ('nb', clf1),('lr', clf3),('mlp',clf4)], voting='hard').fit(Xtrain, Ytrain1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_ite

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/Workspace/hasoc/testpSet.csv')

In [ ]:
test

,_id,features
0,60c5d6bf5659ea5e55deffcb,Fewer people coming in for vaccinations So sad...
1,60c5d6bf5659ea5e55df028c,MattHancock This may all be true But What did ...
2,60c5d6bf5659ea5e55def377,Layla EFC ve unfollowed him the wanker
3,60c5d6bf5659ea5e55def4c7,You guys are losing it all over the world The ...
4,60c5d6bf5659ea5e55df01a6,And thus death laughs It is sad merriment But ...
...,...,...
1276,60c5d6bf5659ea5e55def74c,AJEnglish Shameless Hinduphobia and bigotry ca...
1277,60c5d6bf5659ea5e55df0380,DonVocero Real Motherfucker
1278,60c5d6bf5659ea5e55defaeb,Gunna was in cunt mode on DS
1279,60c5d6bf5659ea5e55df03c3,roomorgue adore all my hooker friends includi...


In [ ]:
X_test = test.tweet.values
# y_test = test.new_label.values

In [ ]:
Xtest = vectorizer.fit_transform(X_test)
# Ytest = y_test

Numer of samples with no words found: 1 / 1280


In [ ]:
y_pred=clf6.predict(Xtest)

In [ ]:
y_pred

array([0, 1, 1, ..., 1, 0, 1])

In [ ]:
predictions=label_encoder.inverse_transform(y_pred)

In [ ]:
p=pd.DataFrame(y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test.label, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.70      0.65       482
           1       0.80      0.73      0.77       798

    accuracy                           0.72      1280
   macro avg       0.71      0.72      0.71      1280
weighted avg       0.73      0.72      0.72      1280

